In [ ]:
from google.colab import files
files.upload()

In [ ]:
!ls -al

In [ ]:
import cv2
import numpy as np
import IPython
from google.colab.patches import cv2_imshow

In [ ]:
file_name = 'test_video.mp4'

In [ ]:
cap = cv2.VideoCapture(file_name)

width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
print("size: {0} x {1}".format(width, height))

# 영상 저장을 위한 VideoWriter 인스턴스 생성
fourcc = cv2.VideoWriter_fourcc(*'XVID')
writer = cv2.VideoWriter('test.avi', fourcc, 24, (int(width), int(height)))

In [ ]:
def detectAndDisplay(frame):
    IPython.display.clear_output(wait=True)
    # Conver the image to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    height, width = gray.shape
    # GaussianBlur for refucing noise
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    # Canny Edge detection
    canny = cv2.Canny(blur, 40, 130)
    mask = np.zeros((height, width), dtype='uint8')
    poly_heigh = int(0.60 * height)
    poly_left = int(0.47 * width)
    poly_right = int(0.53 * width)
    polygons = np.array([[(0, height), (poly_left, poly_heigh), (poly_right, poly_heigh), (width, height)]])
    cv2.fillPoly(mask, polygons, 255)
    masked = cv2.bitwise_and(canny, mask)
    lines = cv2.HoughLinesP(masked, 2, np.pi / 180, 20, np.array([]), 20, 10)
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 5)
            
    cv2_imshow(frame)

In [ ]:
#-- 2. Read the video stream
#cap = cv2.VideoCapture(file_name)

if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)

while True:
    ret, frame = cap.read()
    if frame is None:
        print('--(!) No captured frame -- Break!')
        break   
    detectAndDisplay(frame)  
    writer.write(frame)

cap.release()
writer.release()  # 저장 종료      

In [ ]:
def download(path):
    try:
        from google.colab import files
        files.download(path)
    except ImportError:
        import os
        print('Error download: ', os.path.join(os, getcwd(), path))

In [ ]:
download('test.avi')